In [ ]:
!pip install fastapi uvicorn pyngrok


In [ ]:
!pip install ragatouille

In [ ]:
!unzip fine_tuned_tinybert.zip


In [12]:
import os
import zipfile

# Define the path where the index should be located
index_path = ".ragatouille/colbert/indexes/my_custom_index"

# Ensure the directory exists
os.makedirs(index_path, exist_ok=True)

# Unzip the file
zip_file_path = 'my_custom_index.zip'


In [13]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for member in zip_ref.namelist():
        filename = os.path.basename(member)
        if filename:
            # Join the correct destination path
            source = os.path.join(index_path, filename)
            with open(source, 'wb') as f:
                f.write(zip_ref.read(member))


In [14]:
!ls .ragatouille/colbert/indexes/my_custom_index


0.codes.pt	 avg_residual.pt  collection.json	   ivf.pid.pt	       plan.json
0.metadata.json  buckets.pt	  docid_metadata_map.json  metadata.json
0.residuals.pt	 centroids.pt	  doclens.0.json	   pid_docid_map.json


In [15]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_index(".ragatouille/colbert/indexes/my_custom_index")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [17]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from pyngrok import ngrok
import uvicorn
import nest_asyncio


In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the fine-tuned TinyBERT model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_tinybert")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_tinybert")

# Set the device (use GPU if available)
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [19]:
app = FastAPI()


In [20]:
# Load the fine-tuned TinyBERT model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_tinybert")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_tinybert")


In [21]:
# Set the device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [22]:
# Define the schema for incoming requests
class QueryRequest(BaseModel):
    query: str

def get_related_articles(query):
    # Use the RAG model to search for related documents
    results = RAG.search(query)

    # Extract and return the UUIDs of the related documents
    article_uuids = [result['document_id'] for result in results]

    return article_uuids


# Function to classify the query as toxic or non-toxic
def classify_query(query: str):
    # Tokenize the input query and move tensors to the correct device
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    # Perform inference
    outputs = model(**inputs)

    # Get the predicted class (0 for non-toxic, 1 for toxic)
    logits = outputs.logits
    prediction = logits.argmax(-1).item()

    return "toxic" if prediction == 1 else "non-toxic"


In [23]:
# Define the main API endpoint for validating the query
@app.post("/validate-query/")
async def validate_query(query: QueryRequest):
    # Classify the query as toxic or non-toxic
    toxicity = classify_query(query.query)

    if toxicity == "toxic":
        return {"message": "Query is toxic, no articles returned."}
    else:
        # Retrieve related articles if the query is non-toxic
        related_articles = get_related_articles(query.query)
        return {"message": "Query is non-toxic.", "related_articles": related_articles}


In [ ]:
TOKEN = ""

In [24]:
!ngrok config add-authtoken TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Allow asynchronous loops in Colab
nest_asyncio.apply()

# Open a tunnel on port 8000 to allow external access using ngrok
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")


In [ ]:
# Run the FastAPI app with Uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)
